In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
38,application_1607211657348_0040,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
from pyspark.sql.functions import array, coalesce, concat,  col

## Placeholder

In [3]:
import hsfs
from hops import hdfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [4]:
node_fg = fs.get_training_dataset("node_td", 1)
node_labels_df = node_fg.read().select("id","is_sar")

In [5]:
node_labels_df.show(5)

+----+------+
|  id|is_sar|
+----+------+
|7605|     1|
|6075|     1|
|5814|     1|
|9360|     1|
|4626|     1|
+----+------+
only showing top 5 rows

## Placeholder

In [6]:
node_embeddings_df = spark.read.csv("hdfs:///Projects/{}/Resources/node_embeddings_features_2_2_16.csv".format(hdfs.project_name(),inferSchema=True,header=False))

In [7]:
feature_names = node_embeddings_df.columns
feature_names = ["_" + s + "c"  for s in feature_names]
feature_names[0]= 'id'
node_embeddings_node_embeddings_df = node_embeddings_df.toDF(*feature_names)

In [8]:
feature_names.remove('id')

In [9]:
emb_td = node_embeddings_node_embeddings_df.join(node_labels_df, ['id'])

In [10]:
emb_td = emb_td.drop("id").withColumn("embedding", array(feature_names)).select("is_sar","embedding").withColumnRenamed("is_sar","target")

## Placeholder

In [11]:
non_sar_df = emb_td.where(col("target")==0)

In [12]:
sar_df = emb_td.where(col("target")==1)

In [13]:
# Now that the data has been prepared, let's split the dataset into a training and test dataframe
[non_sar_train_df, non_sar_eval_df] = non_sar_df.randomSplit([0.8, 0.02],seed = 12345)

In [18]:
non_sar_td = fs.create_training_dataset(name="gan_non_sar_training_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["target"], 
                                       statistics_config=False, 
                                       description="non sar dataset for gan training")
non_sar_td.save(non_sar_train_df)

## Placeholder

In [15]:
eval_df = non_sar_eval_df.union(sar_df)

In [20]:
gan_eval_ds = fs.create_training_dataset(name="gan_eval_df",
                                       version=1,
                                       data_format="tfrecord",
                                       label=["target"], 
                                       statistics_config=False, 
                                       description="evaluation dataset for gan training")
gan_eval_ds.save(eval_df)